# Training Models at Scale with AI Platform

**Learning objectives**

1. Learn how to organise your training code into a Python package
2. Train your model using a cloud infrastructure via Google Cloud AI Platform Training
3. Learn how to run your training package using Docker containers and push training Docker images to a Docker registry

# Introduction

In this notebook we'll make the jump from training locally to training in the cloud. We'll take advantage of Google Cloud's [AI Platform Training](https://cloud.google.com/ai-platform/).

AI Platform Training is a managed service that allows the training and deployment of ML models without having to provision or maintain servers. The infrastructure is handled seamlessly by the managed services.

In [ ]:
!pip install --user google-cloud-bigquery=1.25.0

In [ ]:
import os

from google.cloud import bigquery

In [ ]:
BUCKET = "<BUCKET>"
PROJECT = "<PROJECT>"
REGION = "<YOUR REGION>"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

# Store the values of `BUCKET`, `OUTDIR`, `PROJECT`, `REGION` and `TFVERSION` in environment variables
os.environ["BUCKET"] = BUCKET
os.environ["OUTDIR"] = OUTDIR
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.1"

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Create BigQuery tables

In [ ]:
# Create a BigQuery dataset for our data
bq = bigquery.Client(project=PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

try:
    bq.create_dataset(dataset)
    print("Dataset created")
except:
    print("Dataset already exists")

Let's create a table with 1 million examples

In [ ]:
%%bigquery

# Creating the table in our dataset
CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amout + fare_amount) AS fare_amount,
    pickup_datetime,
    pikcup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    "unusued" AS key
FROM
    `nyc-tlc.yellow.trips`
WHERE
    ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
AND
    fare_amount >= 2.5
AND
    pickup_longitude > -78
AND
    pickup_longitude < -70
AND
    dropoff_longitude > -78
AND
    dropoff_longitude < -70
AND
    pickup_latitude > 37
AND
    pickup_latitude < 45
AND
    dropoff_latitude > 37
AND
    dropoff_latitude < 45
AND
    passenger_count > 0

Make the validation data set be 1/10 the size of the training data set.

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    "unusued" AS key
FROM
    `nyc-tlc.yellow.trips`
WHERE
    ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
AND
    fare_amount >= 2.5
AND
    pickup_longitude > -78
AND
    pickup_longitude < -70
AND
    dropoff_longitude > -78
AND
    dropoff_longitude < -70
AND
    pickup_latitude > 37
AND
    pickup_latitude < 45
AND
    dropoff_latitude > 37
AND
    dropoff_latitude < 45
AND
    passenger_count > 0

## Export the tables as CSV files

In [ ]:
%%bash

# Deleting the contents of the output directory
echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

# Fetching the training data to output directory
echo "Extracting training data to $OUTDIR"
bq --location=US extract \
    --destination_format CSV \
    --field_delimiter "," --noprint_header \
    taxifare.feateng_training_data \
    $OUTDIR/taxi-train-*.csv
    
echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
    --destination_format CSV \
    --field_delimiter "," --noprint_header \
    taxifare.feateng_valid_data \
    $OUTDIR/taxi-valid-*.csv
    
# The `ls` command will show the content of working directory
gsutil ls -l $OUTDIR

In [ ]:
# The `cat` command will output the contents of one or more URLs
# Using `head -2` we are showing only top two output files
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

## Make code compatible with AI Platform Training Service

In order to make our code compatible with AI Platform Training we need to make the following chages:

1. Upload data to Google Cloud Storage
2. Move code into a trainer Python package
3. Submit training job with `gcloud` to train on AI Platform

### Upload data to Google Cloud Storage (GCS)

Cloud services don't have access to our local files, so we need to upload them to a location the Cloud servers can read from. In this case we'll use GCS.

In [ ]:
# The `ls` command will show the content of working directory
!gsutil ls gs://$BUCKET/taxifare/data

### Move code into a Python package

The first thing to do is to convert your training code snippets into a regular Python package that we will then `pip install` into the Docker container.

A Python package is simply a collection of one or more `.py` files along with an `__init__.py` file to identify the containing directory as a package. The `__init__.py` sometimes contains initialisation code but for our purposes an empty file suffices.

#### Create the package directory

Our package directory contains $3$ files:

In [9]:
mkdir taxifare/

mkdir: cannot create directory ‘taxifare/’: File exists


In [13]:
cd taxifare

/home/antounes/Documents/tensorflow/art-and-science-of-ml/taxifare


In [14]:
mkdir trainer

In [ ]:
cd ..

In [20]:
ls taxifare/trainer/

model.py


#### Paste existing code into `model.py`

A Python package requires our code to be in a `.py` file, as opposed to notebook cells. 

In [40]:
%%writefile taxifare/trainer/model.py
import datetime
import logging
import os
import shutil

import numpy as np
import tensorflow as tf

from tensorflow.keras import activations
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow import feature_column as fc

logging.info(tf.__version__)

# Defining the feature names into a list `CSV_COLUMNS`
CSV_COLUMNS = [
    "fare_amount",
    "pickup_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
    "key"
]
LABEL_COLUMN = "fare_amount"
# Defining the default values into a list `DEFAULTS`
DEFAULTS = [[0.0], ["na"], [0.0], [0.0], [0.0], [0.0], [0.0], ["na"]]
DAYS = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]

def features_and_labels(row_data):
    for unwanted_col in ["key"]:
        row_data.pop(unwanted_col)
    # The `.pop()` method will return item and drom from DataFrame
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label

def load_dataset(pattern, batch_size, num_repeat):
    # The `tf.data.experimental.make_csv_dataset()` method reads CSV files into a `tf.data.Dataset`
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=pattern,
        batch_size=batch_size,
        column_names=CSV_COLUMNS,
        column_defaults=DEFAULTS,
        num_epochs=num_repeat
    )
    # The `.map()` function executes a specified function for each item in an iterable
    # The item is sent to the function as a parameter
    return dataset.map(features_and_labels)

def create_train_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=None)
    # The `.prefetch()` method will start a background thread to populate an ordered buffer that acts
    # like a queue, so that downstream pipeline stages aren't paused
    return dataset.prefetch(1)

def create_eval_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=1)
    return dataset.prefetch(1)

def parse_datetime(s):
    if type(s) is not str:
        s = s.numpy().decode("utf-8")
    return datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S %Z")

def euclidean(params):
    lon1, lat1, lon2, lat2 = params
    londiff = lon2 - lon1
    latdiff = lat2 - lat1
    return tf.sqrt(londiff ** 2 + latdiff ** 2)

def get_dayofweek(s):
    ts = parse_datetime(s)
    return DAYS[ts.weekday()]

@tf.function
def dayofweek(ts_in):
    return tf.map_fn(
        lambda s: tf.py_function(get_dayofweek, inp=[s], Tout=tf.string),
        ts_in
    )

@tf.function
def fare_thresh(x):
    return 60 * tf.activations.relu(x)

def transform(inputs, NUMERIC_COLS, STRING_COLS, nbuckets):
    # Pass-through columns
    transformed = inputs.copy()
    del transformed["pickup_datetime"]
    
    feature_columns = {
        colname: tf.feature_column.numeric_column(colname)
        for colname in NUMERIC_COLS
    }
    
    # Scaling longitude from range [-70, -78] to [0, 1]
    for lon_col in ["pickup_longitude", "dropoff_longitude"]:
        transformed[lon_col] = tf.keras.layers.Lambda(
            lambda x: (x + 78)/8.0,
            name="scale_{}".format(lon_col)
        )(inputs[lon_col])
        
    # Scaling latitude from range [37, 45] to [0, 1]
    for lat_col in ["pickup_latitude", "dropoff_latitude"]:
        transformed[lat_col] = tf.keras.layers.Lambda(
            lambda x: (x - 37)/8.0,
            name="scale_{}".format(lat_col)
        )(inputs[lat_col])
        
    # Adding Euclidean distance (no need to be accurate: NN will calibrate it)
    transformed["euclidean"] = tf.keras.layers.Lambda(euclidean, name="euclidean")([
        inputs["pickup_longitude"],
        inputs["pickup_latitude"],
        inputs["dropoff_longitude"],
        inputs["dropoff_latitude"]
    ])
    feature_columns["euclidean"] = tf.feature_column.numeric_column("euclidean")
    
    # Hour of day from timestamp of form `2010-02-08 09:17:00+00:00`
    transformed["hourofday"] = layers.Lambda(
        lambda x: tf.strings.to_number(
            tf.strings.substr(x, 11, 2), out_type=tf.dtypes.int32),
        name="hourofday"
    )(inputs["pickup_datetime"])
    feature_columns["hourofday"] = tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_identity(
            "hourofday", num_buckets=24))
    
    latbuckets = np.linspace(0, 1, nbuckets).tolist()
    lonbuckets = np.linspace(0, 1, nbuckets).tolist()
    b_plat = tf.feature_column.bucketized_column(
        feature_columns["pickup_latitude"], latbuckets)
    b_dlat = tf.feature_column.bucketized_column(
        feature_columns["dropoff_latitude"], latbuckets)
    b_plon = tf.feature_column.bucketized_column(
        feature_columns["pickup_longitude"], lonbuckets)
    b_dlon = tf.feature_column.bucketized_column(
        feature_columns["dropoff_longitude"], lonbuckets)
    ploc = tf.feature_column.crossed_column(
        [b_plat, b_plon], nbuckets**2)
    dloc = tf.feature_column.crossed_column(
        [b_dlat, b_dlon], nbuckets**2)
    pd_pair = tf.feature_column.crossed_column([ploc, dloc], nbuckets**4)
    feature_columns["pickup_and_dropoff"] = tf.feature_column.embedding_column(
        pd_pair, 100)
    
    return transformed, feature_columns

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

def build_dnn_model(nbuckets, nnsize, lr):
    # Input layer is all float except for `pickup_datetime` which is a string
    STRING_COLS = ["pickup_datetime"]
    NUMERIC_COLS = (
        set(CSV_COLUMNS) - set([LABEL_COLUMN, "key"]) - set(STRING_COLS)
    )
    inputs = {
        colname: layers.Input(name=colname, shape=(), dtype="float32")
        for colname in NUMERIC_COLS
    }
    inputs.update({
        colname: layers.Input(name=colname, shape=(), dtype="string")
        for colname in STRING_COLS
    })
    
    # Transforms
    transformed, feature_columns = transform(
        inputs, NUMERIC_COLS, STRING_COLS, nbuckets=nbuckets
    )
    dnn_inputs = tf.keras.layers.DenseFeatures(feature_columns.values())(transformed)
    
    x = dnn_inputs
    for layer, nodes in enumerate(nnsize):
        x = layers.Dense(nodes, activation="relu", name="h{}".format(layer))(x)
    output = tf.keras.layers.Dense(1, name="fare")(x)
    
    model = tf.keras.models.Model(inputs, output)
    lr_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=lr_optimizer, loss="mse", metrics=[rmse, "mse"])
    
    return model

def train_and_evaluate(hparams):
    batch_size = hparams["batch_size"]
    nbuckets = hparams["nbuckets"]
    lr = hparams["lr"]
    nnsize = hparams["nnsize"]
    eval_data_path = hparams["eval_data_path"]
    num_evals = hparams["num_evals"]
    num_examples_to_train_on = hparams["num_examples_to_train_on"]
    output_dir = hparams["output_dir"]
    train_data_path = hparams["train_data_path"]
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    savedmodel_dir = os.path.join(output_dir, "export/savedmodel")
    model_export_path = os.path.join(savedmodel_dir, timestamp)
    checkpoint_path = os.path.join(output_dir, "checkpoints")
    tensorboard_path = os.path.join(output_dir, "tensorboard")
    
    if tf.io.gfile.exists(output_dir):
        tf.io.gfile.rmtree(output_dir)
        
    model = build_dnn_model(nbuckets, nnsize, lr)
    logging.info(model.summary())
    
    trainds = create_train_dataset(train_data_path, batch_size)
    evalds = create_eval_dataset(eval_data_path, batch_size)
    
    steps_per_epoch = num_examples_to_train_on // (batch_size * num_evals)
    
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        save_weights_only=True,
        verbose=1
    )
    tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_path)
    
    history = model.fit(
        trainds,
        validation_data=evalds,
        epochs=num_evals,
        steps_per_epoch=max(1, steps_per_epoch),
        verbose=2, #0=silent, #1=progress bar, 2=one line per epoch
        callbacks=[checkpoint_cb, tensorboard_cb]
    )
    
    # Exporting the model with default serving function
    tf.saved_model.save(model, model_export_path)
    return history

Overwriting taxifare/trainer/model.py


## Modify code to read data from and write checkpoint files to GCS

If you look closely above, you'll notice a new function, `train_and_evaluate` that wraps the code that actually trains the model. This allows us to parametrise the training by passing a dictionary of parameters to this function (e.g., `batch_size`, `num_examples_to_train_on`, `train_data_path`, etc.)

This is useful because the output directory, data paths and number of train steps will be different depending on whether we're training locally or in the cloud. Parameterising allows us to use the same code for both.

We specify these parameters at runtime via the command line. Which means we need to add code to parse command line parameters and invoke `train_and_evaluate()` with these parameters. This is the job of the `task.py` file.

In [41]:
%%writefile taxifare/trainer/task.py

# The argparse module makes it easy to write user-friendly command-line interfaces. It parses the defined arguments
# from the `sys.argv`
# The argparse module also automatically generates help & usage messages and issues errors when users give the 
# program invalid arguments
import argparse

from trainer import model

# Write a `task.py` file for adding code to parse command line parameters and invoke `train_and_evaluate()` with
# these parameters
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--batch_size",
        help="Batch size for training steps",
        type=int,
        default=32
    )
    parser.add_argument(
        "--eval_data_path",
        help="GCS location pattern of eval files",
        required=True
    )
    parser.add_argument(
        "--nnsize",
        help="Hidden layer sizes (provide space-separated sizes)",
        nargs="+",
        type=int,
        default=[32, 8]
    )
    parser.add_argument(
        "--nbuckets",
        help="Number of buckets to divide lat and lon with",
        type=int,
        default=10
    )
    parser.add_argument(
        "--lr",
        help = "learning rate for optimizer",
        type=float,
        default=0.001
    )
    parser.add_argument(
        "--num_evals",
        help="Number of times to evaluate model on eval data",
        type=int,
        default=5
    )
    parser.add_argument(
        "--num_examples_to_train_on",
        help="Number of examples to train on",
        type=int,
        default=100
    )
    parser.add_argument(
        "--output_dir",
        help="GCS location to write checkpoints and export models",
        required=True
    )
    parser.add_argument(
        "--train_data_path",
        help="GCS location pattern of train files containing eval URLs",
        required=True
    )
    parser.add_argument(
        "--job-dir",
        help="This model ignores this field, but it is required by gcloud",
        default="junk"
    )
    args = parser.parse_args()
    hparams = args.__dict__
    hparams.pop("job-dir", None)
    
    model.train_and_evaluate(hparams)

Overwriting taxifare/trainer/task.py


## Run trainer module package locally

Now we can test our training code locally as follows using the local test data. We'll run a very small training job over a single file with a small batch size and one eval step.

In [42]:
%%bash

# Testing our training code locally
EVAL_DATA_PATH=data/taxi-valid*
TRAIN_DATA_PATH=data/taxi-train*
OUTPUT_DIR=models

test ${OUTPUT_DIR} && rm -rf ${OUTPUT_DIR}
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare

python3 -m trainer.task \
--eval_data_path $EVAL_DATA_PATH \
--output_dir $OUTPUT_DIR \
--train_data_path $TRAIN_DATA_PATH \
--batch_size 5 \
--num_examples_to_train_on 100 \
--num_evals 1 \
--nbuckets 10 \
--lr 0.001 \
--nnsize 32 8

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dropoff_latitude (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
dropoff_longitude (InputLayer)  [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_latitude (InputLayer)    [(None,)]            0                                            
______________________________________________________________________________________________

2021-04-09 22:57:45.278832: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-04-09 22:57:45.278868: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-04-09 22:57:46.900605: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-09 22:57:46.900785: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-04-09 22:57:46.900801: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-04-09 22:57:46.900824: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

## Run your training package on Cloud AI Platform

Once the code works in standalone mode locally, you can run it on Cloud AI Platform. To submit to the CLoud we use `gcloud ai-platform jobs submit training [jobname]` and simply specify some parameters for AI Platform Training:


- jobid: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- region: Cloud region to train in

The arguments before `-- \` are AI Platform Training. The arguments after `-- \` are sent to our `task.py`.

Because this is on the entire data set, it will take a while. You can monitor the job from the GCP console in the Cloud AI Platform section

In [ ]:
%%bash

# Output directory and jobID
OUTDIR=gs://${BUCKET}/taxifare/trained_model_$(date -u +%y%m%d_%H%M%S)
JOBID=taxifare_$(date -y +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBID}
gsutil -m rm -rf ${OUTDIR}

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=100
NUM_EVALS=100
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# GCS paths
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

gcloud ai-platform jobs submit training $JOBID \
    --module-name=trainer.task \
    --package-path=taxifare/trainer \
    --staging-bucket=gs://${BUCKET} \
    --python-version=3.7 \
    --runtime-version=${TFVERSION} \
    --region=${REGION} \
    -- \
    --eval_data_path $EVAL_DATA_PATH \
    --output_dir $OUTDIR \
    --train_data_path $TRAIN_DATA_PATH \
    --batch_size $BATCH_SIZE \
    --num_examples_to_train_on $NUM_EXAMPLES_TO_TRAIN_ON \
    --num_evals $NUM_EVALS \
    --nbuckets $NBUCKETS \
    --lr $LR \
    --nnsize $NNSIZE

## Run your training package using a Docker container

AI Platform Training also supports training in custom containers, allowing users to bring their own Docker containers with any pre-installed ML framework or algorithm to run on AI Platform Training.

In this last section, we'll see how to submit a Cloud training job using a customised Docker image.

Containerising your `taxifare/trainer` package involves 3 steps:

- Writing a `Dockerfile` in `/taxifare`
- Buidling the Docker image
- Pushing it to the Google Cloud container registry in your GCP project

The `Dockerfile` specifies

1. How the container needs to be provisioned so that all the dependencies in our code are satisfied
2. Where to copy our trainer Package in the container and how to install it (`pip install /trainer`)
3. What command to run when the container is ran (the `ENTRYPOINT` line)

In [43]:
%%writefile taxifare/Dockerfile
# Writing the Dockerfile
FROM gcr.io/deeplearting-platform-release/tf2-cpu

COPY . /code

WORKDIR /code

ENTRYPOINT ["python3", "-m", "trainer.task"]

Writing taxifare/Dockerfile


In [ ]:
!gclud auth configure-docker

In [ ]:
%%bash

# Pushing the docker image to Google Cloud container registry in your GCP project
PROJECT_DIR=$(cd taxifare && pwd)
PROJECT_ID=$(gcloud config list project --format "value(core.project)")
IMAGE_NAME=taxifare_training_container
DOCKERFILE=$PROJECT_DIR/Dockerfile
IMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_NAME

docker build $PROJECT_DIR -f $DOCKERFILE -t $IMAGE_URI

docker push $IMAGE_URI

## Train using a custom container on AI Platform

To submit to the Cloud we use `gcloud ai-platform jobs submit training [jobname]` and simply specify some additional parameters for AI Platform Training:

- jobname: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- master-image-uri: The URI of the Docker image we pushed in the Google Cloud registry
- region: Cloud region to train in

The arguments before `-- \` are for AI Platform Training. The arguments after `-- \` are sent to our `task.py`

You can track your job and view logs using [cloud console](https://console.cloud.google.com/mlengine/jobs)

In [ ]:
%%bash

PROJECT_ID=$(gcloud config list project --format "value(core.project)")
BUCKET=$PROJECT_ID
REGION="us-central1"

# Output directory and jobID
OUTDIR=gs://${BUCKET}/taxifare/trained_model
JOBID=taxifare_container_$(date -u + %y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBID}
gsutil -m rm -rf ${OUTDIR}

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=100
NUM_EVALS=100
NBUCKETS=10
NNSIZE="32 8"

# AI Platform machines to use for training
MACHINE_TYPE=n1-standard-4
SCALE_TIER=CUSTOM

# GCS paths
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

IMAGE_NAME=taxifare_training_container
IMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_NAME

gcloud beta ai-platform jobs submit training $JOBID \
    --staging-bucket=gs://$BUCKET \
    --region=$REGION \
    --master-image-uri=$IMAGE_URI \
    --master-machine-type=$MACHINE_TYPE \
    --scale-tier=$SCALE_TIER \
    -- \
    --eval_data_path $EVAL_DATA_PATH \
    --output_dir $OUTDIR \
    --train_data-path $TRAIN_DATA_PATH \
    --batch_size $BATCH_SIZE \
    --num_examples_to_trian_on $NUM_EXAMPLES_TO_TRAIN_ON \
    --num_evals $NUM_EVALS \
    --nbuckets $NBUCKET \
    --nnsize $NNSIZE